### Kernel Learning

- Utilizes C-Support Vector Classification in Scikit-Learn
https://scikit-learn.org/stable/modules/svm.html


- Dependency is libsvm

- Multiclass is handled within a 1 vs. 1 approach

In [2]:
import sklearn
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score,precision_score,recall_score

import numpy as np
import pandas as pd

/home/jbohn/miniconda3/envs/research/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load and Split Data

In [3]:
label_data=pd.read_csv('/home/jbohn/jupyter/personal/sbu/Kernel_Learning/data/labeled_data.csv',index_col=0)


In [4]:
features=label_data[['FB0','FA0','FB2','FA2']].copy()
outcomes=label_data['outcome'].copy()
X_train , X_test , y_train, y_test = train_test_split(features, outcomes, random_state=1234,train_size=0.75)

### Normalize features 

- Linear SVM aims to maximize distance between observations, therefore we wish to normalize each feature such that there is an equal influence

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

array([[-0.30855379, -0.29611603, -0.00730284,  0.19584904],
       [-0.30855379,  1.20483414, -0.00730284,  1.36859778],
       [-0.30855379, -0.10849725, -0.00730284,  0.19584904],
       ...,
       [-0.51947924, -0.4837348 , -0.00730284,  0.04925545],
       [-0.51947924, -0.4837348 , -0.44799169,  0.04925545],
       [-0.51947924, -0.4837348 , -2.21074709,  0.04925545]])

### Single Kernel Training

Radial Basis Function of scaled gamma param (CV)

Evaluation Metrics (Multilabel as 3 outcomes)
- Raw Accuracy: % of correct observations
- Precision: Ratio of correct positive predictions to all outcomes
- Recall : Ratio correct positive predictions to potential

In [22]:
clf = svm.SVC(kernel='rbf',gamma='scale',verbose=False,decision_function_shape='ovo')
model=clf.fit(X_train,y_train)

Examining the decision function of our classifier

In [24]:
sample=X_test.iloc[0].values.reshape(1,-1)

model.decision_function(sample)

/home/jbohn/miniconda3/envs/research/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([[-0.98412853, -0.4796191 ,  0.97505275]])

### Cross Validation of Kernel Bandwidth



In [26]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted', zero_division=0),
           'recall': make_scorer(recall_score, average = 'weighted', zero_division=0)}

# weighted average between multilabeled outcomes 

In [27]:
cv_results = cross_validate(clf, X_train, y_train, cv=10,scoring=scoring,return_train_score=True)

In [28]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall
0,0.051719,0.008779,0.409091,0.419314,0.167355,0.683516,0.409091,0.419314
1,0.048136,0.007641,0.409091,0.418043,0.167355,0.759319,0.409091,0.418043
2,0.072835,0.009625,0.409091,0.419314,0.167355,0.683516,0.409091,0.419314
3,0.046446,0.008017,0.409091,0.419314,0.167355,0.683516,0.409091,0.419314
4,0.052648,0.008361,0.397727,0.418043,0.164577,0.759319,0.397727,0.418043
5,0.066541,0.010641,0.413793,0.418782,0.171225,0.683382,0.413793,0.418782
6,0.044314,0.011183,0.413793,0.421320,0.457742,0.649414,0.413793,0.421320
7,0.041616,0.007822,0.402299,0.421320,0.170385,0.672806,0.402299,0.421320
8,0.062684,0.012026,0.413793,0.420051,0.171225,0.672950,0.413793,0.420051
9,0.048322,0.009768,0.402299,0.421320,0.161844,0.759625,0.402299,0.421320


### Out of Sample Accuracy

In [29]:
model.score(X_test,y_test)

0.3801369863013699

### SimpleMKL Training

- In scikit-learn we utilized a single kernel (radial basis function) with cross validated bandwidth (via grid search)

- We now want to extend to the case of multiple kernels (linear combination of a basis set)

- Utilize this function https://github.com/qintian0321/SimpleMKL_python/blob/master/mysimpleMKL_implementation.py